# 빅데이터 분석기사 실기 준비
## 프리렉 출판사 빅데이터 분석기사 실기 책 자료 참고

## 데이터 탐색

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/bigData-main/mtcars.csv')
print(df.head())

mpg : 연비
Unnamed : 0 : 차 모델 명
cyl : 엔진 기통
disp : 배기량
hp : 마력
drat : 뒤 차축비
wt : 무게
qsec : 1/4 mile 도달 시간
vs : V형 엔진/ 직렬엔진
am : 변속기 
gear : 전진기어 개수
carb : 기화기 개수

In [ ]:
print(df.shape)

In [ ]:
print(type(df))

In [ ]:
print(df.columns)

In [ ]:
print(df.describe())

cyl - 실린더가 50기통임은 이상치로 여겨짐

In [ ]:
print(df['am'].unique())
print(df['gear'].unique())
print(df['vs'].unique())

In [ ]:
print(df.info())

In [ ]:
print(df.corr())

종속 변수와 독립 변수 구분 \
mpg를 종속변수로, 외 열은 독립 변수

In [ ]:
x = df.drop(columns = 'mpg')
y = df['mpg']

In [ ]:
print(x.head())
print(x.columns)

In [ ]:
print(y.head())

## 데이터 전처리

In [ ]:
# 불필요한 컬럼 처리
print(x.head())
x = x.iloc[:,1:]

In [ ]:
print(x.head())

In [ ]:
# 결측치 처리
x.isnull().sum()

In [ ]:
# cyl, qsec 에만 존재 확인, 평균값으로 대치
x_cyl_mean = x['cyl'].mean()

print(x_cyl_mean)

In [ ]:
x['cyl'] = x['cyl'].fillna(x_cyl_mean)

In [ ]:
# qsec는 중위값으로 대치
x_qsec_median = x['qsec'].median()
print(x_qsec_median)

In [ ]:
x['qsec'] = x['qsec'].fillna(x_qsec_median)

In [ ]:
x.isnull().sum()

In [ ]:
# gear에 잘못된 값 바꾸기
print(x['gear'].unique())

In [ ]:
x['gear'] = x['gear'].replace('*3', '3').replace('*5','5')

In [ ]:
print(x['gear'].unique())

In [ ]:
# 이상값 처리, IQR 처리

# 기초통계량 확인
x_describe = x.describe()
print(x_describe)

In [ ]:
print(x_describe.loc['75%'], x_describe.loc['25%'])

In [ ]:
x_iqr = x_describe.loc['75%'] - x_describe.loc['25%']
print(x_iqr)

In [ ]:
print(x_describe.loc['75%'] + (1.5 * x_iqr))

In [ ]:
print(x_describe.loc['max'])

In [ ]:
#cyl, hp, wt, qsec, carb 는 이상치 존재
# 이외에 cyl은 14 이상이 최대이기에 초과 값은 이상치 처리

# cyl
print(x.loc[x['cyl'] > 14])
x.loc[14,'cyl'] = 14

In [ ]:
# hp
print(x.loc[x['hp'] > 305.25])
x.loc[30,'hp'] = 305.25

In [ ]:
# 최소 경계
print(x_describe.loc['25%'] - (1.5 * x_iqr))

In [ ]:
x_describe.loc['min']

In [ ]:
#표준편차 활용 이상치 처리
#평균 +- 1.5표준편차 범위 밖은 이상치
#함수로 이상치 인덱스 출력
def outlier(data, column):
  mean = data[column].mean()
  std = data[column].std()
  lowest = mean - (1.5 * std)
  highest = mean + (1.5 * std)
  print('lowest : ', lowest, '\n', 'highest : ', highest)
  outlier_index = data[column][(df[column] < lowest) | (data[column] > highest)].index
  return outlier_index

In [ ]:
outlier(x, 'qsec')

In [ ]:
print(x.loc[24,'qsec'])

In [ ]:
x.loc[24,'qsec'] = 42.245
print(x.loc[24,'qsec'])

In [ ]:
print(outlier(x, 'carb'))

In [ ]:
print(x.loc[[29,30], 'carb'])

In [ ]:
x.loc[[29,30], 'carb'] = 5.235

In [ ]:
print(x.loc[[29,30], 'carb'])

## 데이터 스케일링

독립변수 범위를 동일하게 만들어줌으로 머신러닝 결과 왜곡을 막고 특정 독립 변수를 무지하지 않기 위해

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
temp = x[['qsec']]

In [ ]:
scaler = StandardScaler()

In [ ]:
print(scaler.fit_transform(temp))

In [ ]:
qsec_s_scaler = pd.DataFrame(scaler.fit_transform(temp))

In [ ]:
print(qsec_s_scaler.describe())

평균이 0에 근사하고 표준편차는 1에 근사한 정규 분표 형태로 변환되었음

In [ ]:
#최소최대 크기 변환
from sklearn.preprocessing import MinMaxScaler

In [ ]:
temp = x[['qsec']]

In [ ]:
scaler = MinMaxScaler()

In [ ]:
qsec_m_scaler = pd.DataFrame(scaler.fit_transform(temp))

In [ ]:
print(qsec_m_scaler)

In [ ]:
# 로버스트 크기 변환
from sklearn.preprocessing import RobustScaler

temp = x[['qsec']]

In [ ]:
scaler = RobustScaler()

qsec_r_scaler = pd.DataFrame(scaler.fit_transform(temp))

print(qsec_r_scaler)

In [ ]:
print(qsec_r_scaler.describe())

### 데이터 타입 변경

In [ ]:
print(x.info())

In [ ]:
# gear는 수치형이 들어가는 연속형 변수이므로 형 변환
x['gear'] = x['gear'].astype(int)

In [ ]:
x['gear'].dtype

In [ ]:
# 범주형을 수치형으로 인코딩
# 원핫인코딩 pd.get_dummies()로 수행
print(pd.get_dummies(x['am']))


In [ ]:
# 이는 여러 단점으로 불편하니 새로운 열이 늘어나지 않고 1개의 열로 판단 되게 수정
# data 종류가 n개라면 n-1개 열만 새로 생성 , get_dummies(drop_first = True) 로
print(pd.get_dummies(x['am'], drop_first = True))

In [ ]:
# get_dummies() 는 기본적으로 범주형 데이터를 골라 인코딩 처리
# sklearn 에서 OneHotEncoder()도 비슷한 기능을 하지만 전체 변수에 인코딩하는 특징
print(x.info())
print(pd.get_dummies(x, drop_first=True))

In [ ]:
# 라벨 인코딩 - 일련번호 부여 방식으로 점차 숫자 커짐 -> 이 때문에 트리 계열에서만 주로 사용

print(x['am'].head())

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

print(encoder.fit_transform(x['am']))
#am 데이터가 2개라 원핫과 비슷하게 변화

In [ ]:
# 3개 이상에서의 변화를 보기 위한 예시
fruit = ['apple', 'banana', 'grape']
fruit_new = encoder.fit_transform(fruit)
print(fruit, fruit_new)

In [ ]:
# 수동 인코딩 replace
x['am_new'] = x['am'].replace('manual', 0).replace('auto', 1)
print(x.head())

In [ ]:
x = x.drop(columns = 'am')
print(x.head())

In [ ]:
# 파생 변수 만들기
# wt 를 등급 나누기, 평균인 3.3보다 낮으면 0 아니면 1
print(x['wt'] < 3.3)
condition = x['wt'] < 3.3

In [ ]:
x.loc[condition, 'wt_class'] = 0
x.loc[~condition, 'wt_class'] = 1


In [ ]:
print(x[['wt','wt_class']])

In [ ]:
x = x.drop(columns = ['wt'])

In [ ]:
print(x.head())

In [ ]:
# qsec_4 만들기, qsec의 4배로 1mile 도달 시간 변수
x['qsec_4'] = x['qsec'] * 4

In [ ]:
print(x[['qsec', 'qsec_4']])

In [ ]:
x = x.drop(columns = ['qsec'])
print(x.head())

## 학습 데이터로 공부하기
### 모델 생성과 모델 검증

In [ ]:
# 데이터 분리 위한 train_Test_split 함수
from sklearn.model_selection import train_test_split

In [ ]:
#train_test_split에는 첫째 독립변수 데이터 셋, 둘째 종속변수 데이터 셋, 셋째 테스트 데이터로 분리할 비율값을 입력
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, \
                                                    random_state = 10)

In [ ]:
print(x_train.head())

In [ ]:
print(y_train.head())

In [ ]:
print(x_test.head())

In [ ]:
print(y_test.head())

### 모델링(학습에 대한 평가)

#### 종속변수가 연속형 변수인 mpg, 범주형 변수인 am 두 가지 연습

In [ ]:
# 선형회귀
from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(x_train, y_train)
y_train_predicted = model.predict(x_train)
y_test_predicted = model.predict(x_test)


In [ ]:
#y절편
print(model.intercept_)

In [ ]:
#독립변수들의 각 기울기 
print(model.coef_)